# Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import pandas as pd

2025-03-24 21:03:06.228598: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 21:03:07.520326: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 21:03:07.798403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742846587.950773  108097 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742846587.998638  108097 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 21:03:08.645948: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
print(tf.__version__)

2.18.0


In [ ]:
# Test for GPU and determine what GPU we have.
# Modified by student to remove warningen caused on local machine.
IN_COLAB = 'google.colab' in sys.modules

gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    details = tf.config.experimental.get_device_details(gpu_devices[0])
    compute_capability=details.get('compute_capability')
    print("Compute capability:",compute_capability)
    if compute_capability[0]>6:
        print("Turn on mixed_float16")
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
else:
    print("No GPU detected.")

In [ ]:
# Modify GPU memory allocator to try to prevent full GPU memory.
# This can in some cases be counter productive!
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Helper Functions

In [3]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [4]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

# Importa Data

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/seconds_per_frame0.50,overlap0.25,mfccs40_.csv"
df = pd.read_csv(DATASET_PATH, header=None)                     # No header in your format
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float16
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string

In [ ]:
INPUT_SHAPE = (40, 16, 1)
dataset = dataset.reshape(dataset.shape[0], INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2])

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {INPUT_SHAPE}")

Dataset shape: (583554, 40, 16, 1)
Labels shape: (583554,)
Input shape: (40, 16, 1)


In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.2, random_state=42, stratify=labels_set)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Print the lengths of the training, validation, and testing sets.
print(f"Training set length: {len(x_train)}")
print(f"Validation set length: {len(x_val)}")
print(f"Testing set length: {len(x_test)}")

Training set length: 373474
Validation set length: 93369
Testing set length: 116711


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights (based on training data).
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
dist_class_weight = dict(enumerate(class_weight))

In [ ]:
print(f"Classes: {np.unique(labels_set)}")
print(f"Class weights: {dist_class_weight}")

# Print out the amount of each class.
unique, counts = np.unique(y_train, return_counts=True)
print(f"Training class distribution: {dict(zip(unique, counts))}")

Classes: ['Background_noise' 'Car' 'Commercial_vehicles' 'Motorcycle']


In [24]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))
y_val = to_categorical(label_encoder.fit_transform(y_val))

NameError: name 'y_train' is not defined

## Model


### Helper Functions

In [ ]:
def compileModel(model, learning_rate = 0.001):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['categorical_accuracy']
    )
    model.summary()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
    
def printConfusionMatrix(model, x_test, y_test, labels_set):
    y_train_int = np.argmax(y_train, axis=1)  # Convert one-hot to integer labels for y_train
    y_test_int = np.argmax(y_test, axis=1)  # Convert one-hot to integer labels for y_test

    y_pred_prob = model.predict(x_test)

    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert softmax probabilities to class indices

    # Ensure that the labels are in 1D arrays (class indices) before passing to confusion_matrix
    print(f"y_train_int shape: {y_train_int.shape}")
    print(f"y_test_int shape: {y_test_int.shape}")
    print(f"y_pred shape: {y_pred.shape}")

    # Define your class labels (ensure they match the number of classes)
    class_names = np.unique(labels_set)  # Assuming labels_set contains class names

    # Generate confusion matrix
    cm = confusion_matrix(y_test_int, y_pred)

    # Visualize the confusion matrix with proper labels
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Print training history
# Function used for printing two plots, one for accuracy and one for loss.
# To be used with the history object from the fit method
def printHistory(history):
  # Collect the number of epochs run based on the amount of loss value under history.
  epochs = len(history.history['loss'])

  epochrange = range(1, epochs + 1)
  train_acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.plot(epochrange, train_acc, 'bo', label='Training acc')
  plt.plot(epochrange, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy (modell 1)')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

  plt.plot(epochrange, train_loss, 'bo', label='Training loss')
  plt.plot(epochrange, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss (modell 1)')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [ ]:
from keras.callbacks import EarlyStopping

# Fit model function used for convinience when fitting the model multiple times
def fitModel(model, x_train, y_train, x_val, y_val, epochs, doPrintHistory, class_weight, verbose, batch_size):

    # Define early stopping callback.
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        verbose=verbose,
        class_weight=class_weight,
        batch_size=batch_size,
        callbacks=[early_stopping],
    )
    # Print history
    if doPrintHistory: printHistory(history)

### Create, train, compile, test

In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    InputLayer(shape=INPUT_SHAPE),

    Conv2D(filters=32, kernel_size=3, activation="relu", padding='same'),
    MaxPooling2D(),

    Conv2D(filters=32, kernel_size=3, activation="relu", padding='same'),
    MaxPooling2D(),

    # Feature Pooling (Combining Max & Average Pooling)
    Flatten(),

    # Fully Connected Layer
    Dense(32, activation="relu", kernel_regularizer=l2(0.001)),

    # Output Layer (Softmax for multi-class classification)
    Dense(units=len(np.unique(labels_set)), activation="softmax") # Softmax barely increase size.
])

In [74]:
compileModel(model)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 40, 16, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 20, 8, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 20, 8, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 10, 4, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │        40,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,725 (198.14 KB)

 Trainable params: 50,725 (198.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
fitModel(model, x_train, y_train, x_val, y_val, 8, True, dist_class_weight, 1, 32)

In [ ]:
# Evaluate the model on the test set.
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

In [ ]:
printConfusionMatrix(model, x_test, y_test, labels_set)

# Save & Compress model

### Keras (h5) model

In [ ]:
KERAS_MODEL_NAME = "tf_model_mini.h5" # Try using .keras instead.
model.save(KERAS_MODEL_NAME)
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")
keras_model_size = get_file_size(KERAS_MODEL_NAME)

File size: 0.226 Megabytes


### TF Lite model

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")
tflite_file_size = get_file_size(TF_LITE_MODEL_FILE_NAME)

INFO:tensorflow:Assets written to: /tmp/tmpor3gw8sx/assets


INFO:tensorflow:Assets written to: /tmp/tmpor3gw8sx/assets


Saved artifact at '/tmp/tmpor3gw8sx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40, 16, 1), dtype=tf.float32, name='keras_tensor_182')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  139938721834160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716697872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716698224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716705968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716710368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716706672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716709664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716707728: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742847307.799955  108097 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742847307.800758  108097 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 21:15:07.804820: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpor3gw8sx
2025-03-24 21:15:07.805872: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 21:15:07.805887: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpor3gw8sx
2025-03-24 21:15:07.814848: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 21:15:07.868095: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpor3gw8sx
2025-03-24 21:15:07.880267: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 75494 microseconds.


### Compress Model

In [ ]:
# Checkout model.
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

# Evaluate model on test set.
interpreter.allocate_tensors()
input_index = input_details[0]['index']
output_index = output_details[0]['index']
tflite_accuracy = 0
for i in range(len(x_test)):
    input_data = x_test[i]
    input_data = np.array(input_data, dtype=np.float32)
    input_data = np.expand_dims(input_data, axis=0)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    tflite_accuracy += (np.argmax(output) == np.argmax(y_test[i]))
tflite_accuracy = tflite_accuracy / len(x_test)
print("TFLite accuracy: ", tflite_accuracy)

Input Shape: [ 1 40 16  1]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


In [ ]:
INT8_MODEL_FILE_NAME = "tf_lite_int8_model.tflite"

# Load your existing model
model = tf.keras.models.load_model(KERAS_MODEL_NAME)

# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.uint8]

# Ensure that the input and output types are uint8
def representative_dataset_gen():
    for i in range(len(x_test)):
        # Get sample input data as a numpy array in a method of your choosing
        yield [x_test[i]]

converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

# Save the quantized model
with open(INT8_MODEL_FILE_NAME, 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpl1udczw1/assets


INFO:tensorflow:Assets written to: /tmp/tmpl1udczw1/assets


Saved artifact at '/tmp/tmpl1udczw1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40, 16, 1), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  139938716033344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716038272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716078448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716082496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716213392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716211456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938718061968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139938716215504: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742847308.800492  108097 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742847308.800546  108097 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 21:15:08.800779: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpl1udczw1
2025-03-24 21:15:08.801414: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 21:15:08.801431: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpl1udczw1
2025-03-24 21:15:08.806285: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 21:15:08.843990: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpl1udczw1
2025-03-24 21:15:08.854394: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 53616 microseconds.


### Test Quantized Model

In [ ]:
# Load and test the quantized model
interpreter = tf.lite.Interpreter(model_path=INT8_MODEL_FILE_NAME)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random data from x_test
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

# Test the TensorFlow Lite model on random data from x_test
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print("TensorFlow Lite model output:", tflite_results)
print("TensorFlow Lite model output shape:", tflite_results.shape)

# Print file size of the TensorFlow Lite model
convert_bytes(get_file_size(INT8_MODEL_FILE_NAME), "KB")

TensorFlow Lite model output: [[inf inf inf inf inf]]


### Convert Quantized Model into CPP

In [ ]:
INT8_MODEL_CPP_FILE_NAME = "pipeline_int8_model_data.cpp"
!xxd -i {INT8_MODEL_FILE_NAME} > {INT8_MODEL_CPP_FILE_NAME}

In [ ]:
# Print size of pipeline_int8_model_data.cpp
convert_bytes(get_file_size(INT8_MODEL_CPP_FILE_NAME), "KB")

File size: 1236.985 Kilobytes
